<a href="https://colab.research.google.com/github/TruongLeGiaKhanh/IBMDataAnalytics/blob/main/ISHE_Flow_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.8 MB/s eta 0:00:00


In [2]:
pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=d5386d303acd496e2a078a7ff890d9b1287cc4c647f287fc30d8b4b738c76e30
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc


In [25]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit.library import QFT, RZZGate, RZGate, UnitaryGate, DiagonalGate
from qiskit.circuit.library.generalized_gates import Diagonal
from qiskit.quantum_info import random_unitary
from qiskit.circuit import Gate
from qiskit.quantum_info import Statevector

In this lab, we will implement the ISHE flow simulation of the 1 dimensional ISF flow. Note that the only differences between different dimensions are the initial encoding step. The quantum algorithm has 3 subcircuits: Prediction, Normalization, and Gauge transformation.

In [37]:
def discretize_domain_with_binary_vectors(d, n):
    """
    Discretizes the domain [-d, d] and outputs both x_j and their binary vectors (j_0, ..., j_{n-1}).
    """
    num_points = 2**n
    delta_x = (2 * d) / num_points

    j_values = np.arange(num_points)
    x_j = -d + (j_values + 0.5) * delta_x

    binary_vectors = np.zeros((num_points, n), dtype=int)
    for i in range(n):
        binary_vectors[:, i] = (j_values // (2**i)) % 2

    return x_j, binary_vectors

def construct_quaternionic_psi_statevector(d, n_spatial_qubits, psi_coefficients_map, time_t):
    """
    Constructs the state vector |psi> as described in equation (36) using Qiskit.
    It returns the augmented statevector, matching common practice.
    """
    x_points, _ = discretize_domain_with_binary_vectors(d, n_spatial_qubits)

    n_s_and_j_qubits = n_spatial_qubits + 1

    state_amplitudes_original = np.zeros(2**n_s_and_j_qubits, dtype=complex)

    for s_val in range(2):
        for j_idx in range(2**n_spatial_qubits):
            x_j_current = x_points[j_idx]
            psi_coeff = psi_coefficients_map(s_val, x_j_current, time_t)
            combined_index = (s_val << n_spatial_qubits) | j_idx
            state_amplitudes_original[combined_index] = psi_coeff

    initial_psi_statevector = Statevector(state_amplitudes_original)

    original_amplitudes = initial_psi_statevector.data
    original_num_qubits = initial_psi_statevector.num_qubits
    new_num_qubits = original_num_qubits + 1
    new_dimension = 2**new_num_qubits
    augmented_amplitudes = np.zeros(new_dimension, dtype=complex)
    augmented_amplitudes[::2] = original_amplitudes
    augmented_statevector = Statevector(augmented_amplitudes)
    return augmented_statevector

In [27]:
def my_psi_coefficients_func(s_val, binary_j_list, current_time):
    # Example: A simple function where coefficients depend on s_val and spatial index
    # You would replace this with your actual wave function calculation!
    spatial_index = int("".join(map(str, binary_j_list)), 2) # Convert binary list to integer

    if s_val == 0: # Corresponds to first component of quaternion
        return (spatial_index + 1) * np.exp(1j * current_time) + 0.5j
    else: # Corresponds to second component of quaternion
        return (spatial_index * 0.5 - 1) * np.exp(-1j * current_time * 0.5) - 0.2j



**1. Step 1: Prediction**

In [28]:
def apply_phase_operator_pvf(phi_state: Statevector, V_F_func, delta_t, h_reduced, d, n_spatial_qubits):
    """
    Applies the phase operator P(V_F*Delta_t) = exp(-i*Delta_t/h * V_F[x_j'])
    to the spatial components of the state |phi>.

    The input state |phi> is assumed to have the structure |s j_{n-1}...j_0 0>,
    where 's' is the MSB (qubit n_spatial_qubits + 1), j's are the spatial qubits
    (qubits 1 to n_spatial_qubits, representing j_{n-1}...j_0),
    and the last '0' is the LSB ancilla (qubit 0).

    Args:
        phi_state (Statevector): The input state |phi>.
        V_F_func (callable): The potential function V_F(x), takes a float x_j' and returns a float.
        delta_t (float): The time step Delta_t.
        h_reduced (float): The reduced Planck constant h.
        d (float): The half-width of the domain [-d, d].
        n_spatial_qubits (int): The number of qubits used for spatial discretization.

    Returns:
        Statevector: The new state after applying the phase operator.
    """
    n_total_qubits = phi_state.num_qubits
    expected_n_total_qubits = n_spatial_qubits + 1 + 1 # s qubit + spatial qubits + ancilla

    if n_total_qubits != expected_n_total_qubits:
        raise ValueError(f"Input state has {n_total_qubits} qubits, but expected "
                         f"{expected_n_total_qubits} based on n_spatial_qubits={n_spatial_qubits}. "
                         "Check state structure or n_spatial_qubits parameter in the state construction.")

    if n_spatial_qubits <= 0:
        raise ValueError("n_spatial_qubits must be greater than 0 for spatial discretization to apply V_F based on x_j'.")

    # Pre-calculate constants for x_j'
    # delta_x = 2d / 2^n
    delta_x = (2 * d) / (2**n_spatial_qubits)
    # c1 = (-d + Delta_x/2) / (n * Delta_x)
    c1 = (-d + delta_x / 2) / (n_spatial_qubits * delta_x)

    # Create a mutable copy of the amplitudes
    new_amplitudes = np.copy(phi_state.data)

    # Iterate through each basis state's amplitude
    # The index 'k' corresponds to the bit string |q_{N-1} ... q_0>
    # where q_0 is the ancilla, q_1 to q_n_spatial_qubits are j_0 to j_{n-1},
    # and q_{n_spatial_qubits+1} is 's'.
    for k in range(len(new_amplitudes)):
        amplitude = new_amplitudes[k]

        # The operator P(V_F*Delta_t) acts only on the spatial components.
        # It's applied to states where the ancilla qubit (LSB, q_0) is |0>.
        # Amplitudes for states where the ancilla is |1> should be zero if the
        # initial state was |psi> |0>, so we can skip or ensure they remain zero.
        if (k % 2) != 0: # Check if LSB (ancilla) is 1
            continue # Skip, as these amplitudes should ideally be zero anyway

        # If the amplitude is already effectively zero, applying a phase won't change it.
        if np.isclose(amplitude, 0.0):
            continue

        # Extract the decimal value of the spatial component j (j_{n-1}...j_0)
        # This is obtained by right-shifting 'k' by 1 (to remove the LSB ancilla bit)
        # and then masking to get the 'n_spatial_qubits' bits that form 'j'.
        j_decimal_value = (k >> 1) & ((1 << n_spatial_qubits) - 1)

        # Calculate x_j' using the formula: x_j' = Delta_x * (j + c1)
        x_j_prime_value = delta_x * (j_decimal_value + c1)

        # Calculate V_F[x_j']
        V_F_val = V_F_func(x_j_prime_value)

        # Calculate the phase factor: exp(-i*Delta_t/h * V_F[x_j'])
        phase_factor = np.exp(-1j * delta_t / h_reduced * V_F_val)

        # Apply the phase to the amplitude
        new_amplitudes[k] *= phase_factor

    # Create the new Statevector from the modified amplitudes
    result_state = Statevector(new_amplitudes)
    return result_state

def define_pk2dt_gate(phi_state: Statevector, phi_angle: float, theta_angle: float, n_spatial_qubits: int) -> Statevector:
    """
    Applies the P(k^2 dt) operator to the input state |phi> by simulating the described circuit decomposition.
    The operator is (ignoring global phase as per instruction):
    Product_{l=0 to n-1} exp(i * alpha_l * Z_l) *
    Product_{i,j=0 and i>j to n-1} exp(i * beta_ij * Z_i \otimes Z_j)

    It acts on a state with the following qubit mapping (LSB to MSB):
    - q_0: 's' qubit
    - q_1: 'j_0' qubit
    - q_2: 'j_1' qubit
    ...
    - q_n_spatial_qubits: 'j_{n_spatial_qubits-1}' qubit
    - q_{n_spatial_qubits + 1}: 'ancilla' qubit (must be |0> for this operator)

    The ZZ gate (Z_i \otimes Z_j) is implemented via the specified circuit:
    CNOT(j_i, ancilla) -> CNOT(j_j, ancilla) -> RZ(angle) on ancilla -> CNOT(j_j, ancilla) -> CNOT(j_i, ancilla)
    where the RZ angle is -2 * beta_ij to achieve the desired phase.

    Args:
        phi_state (Statevector): The input quantum state.
        phi_angle (float): The angle 'phi' for the global phase term (ignored for now).
        theta_angle (float): The angle 'theta' for the Z and ZZ terms.
        n_spatial_qubits (int): The number of qubits used for spatial discretization (n in formulas).

    Returns:
        Statevector: The state vector after applying the P(k^2 dt) operator.
    """
    if n_spatial_qubits <= 0:
        raise ValueError("n_spatial_qubits must be greater than 0.")

    total_qubits_in_state = phi_state.num_qubits
    expected_qubits = n_spatial_qubits + 2 # spatial (j) + s + ancilla

    if total_qubits_in_state != expected_qubits:
        raise ValueError(f"Input state has {total_qubits_in_state} qubits, but expected {expected_qubits} "
                         f"based on n_spatial_qubits={n_spatial_qubits} and state |ancilla j_{n_spatial_qubits-1}...j_0 s>.")

    new_amplitudes = np.copy(phi_state.data)

    # Global phase factor (phi_angle) is ignored as per instruction.
    # global_phase_angle = phi_angle / (2**(2 * n_spatial_qubits - 3))
    # global_phase_factor = np.exp(1j * global_phase_angle)

    # Iterate through all basis states and apply the combined phase
    for k in range(len(new_amplitudes)):
        amplitude = new_amplitudes[k]

        # If amplitude is effectively zero, skip calculation for this basis state
        if np.isclose(amplitude, 0.0):
            continue

        # Check ancilla qubit (MSB) - it should be 0 for this operator to function as intended on spatial qubits.
        # Qubit index for ancilla is n_spatial_qubits + 1 (MSB)
        ancilla_bit = (k >> (n_spatial_qubits + 1)) & 1
        if ancilla_bit != 0:
            # If the ancilla is |1>, the current model of the operator's application doesn't modify it.
            # Continue to the next amplitude as this operator is implicitly on the |...0> subspace.
            continue

        # --- Calculate phase contribution from single-qubit Z terms ---
        # Product_{l=0 to n-1} exp(i * alpha_l * Z_l)
        # alpha_l = theta_angle / 2^(2n-l-4)
        # Z_l on |j_l> gives (-1)^j_l
        phase_single_Z_product = 1.0
        for l in range(n_spatial_qubits):
            # j_l is mapped to qubit (l+1) in the state vector (q_0 is s, q_1 is j_0, etc.)
            j_l_bit = (k >> (l + 1)) & 1
            alpha_l = theta_angle / (2**(2 * n_spatial_qubits - l - 4))
            phase_single_Z_product *= np.exp(1j * alpha_l * ((-1)**j_l_bit))

        # --- Calculate phase contribution from two-qubit ZZ terms using circuit decomposition ---
        # Product_{i,j=0 and i>j to n-1} exp(i * beta_ij * Z_i \otimes Z_j)
        # beta_ij = theta_angle / 2^(2n-i-j)
        #
        # Circuit: CNOT(j_i, ancilla) -> CNOT(j_j, ancilla) -> RZ(angle) on ancilla -> CNOT(j_j, ancilla) -> CNOT(j_i, ancilla)
        #
        # Derivation of phase:
        # This sequence applies a phase exp(-1j * RZ_angle / 2 * (-1)^(ancilla_bit_after_CNOTs))
        # `ancilla_bit_after_CNOTs` = `initial_ancilla_bit` XOR `j_i_bit` XOR `j_j_bit`.
        # Since `initial_ancilla_bit` is 0 (due to the `if ancilla_bit != 0: continue` check),
        # `ancilla_bit_after_CNOTs` = `j_i_bit` XOR `j_j_bit`.
        #
        # We want the overall effect to be `exp(1j * beta_ij * Z_i Z_j)`
        # `Z_i Z_j` value is `(-1)^(j_i_bit + j_j_bit)` (which is `+1` if bits are same, `-1` if bits are different).
        # Note: `(-1)^(A XOR B)` is `+1` if `A=B`, `-1` if `A!=B`. This matches `(-1)^(A+B)` (mod 2) for boolean inputs.
        # So we want to apply `exp(1j * beta_ij * (-1)^(j_i_bit XOR j_j_bit))`.
        #
        # This implies:
        #   If `j_i_bit == j_j_bit` (XOR is 0): phase = `exp(1j * beta_ij)`
        #     From circuit: `exp(-1j * RZ_angle / 2 * (-1)^0)` = `exp(-1j * RZ_angle / 2)`
        #     So, `beta_ij = -RZ_angle / 2` => `RZ_angle = -2 * beta_ij`
        #   If `j_i_bit != j_j_bit` (XOR is 1): phase = `exp(-1j * beta_ij)`
        #     From circuit: `exp(-1j * RZ_angle / 2 * (-1)^1)` = `exp(1j * RZ_angle / 2)`
        #     So, `-beta_ij = RZ_angle / 2` => `RZ_angle = -2 * beta_ij`
        #
        # Conclusion: The `RZ` angle on the ancilla for each ZZ term should be `-2 * beta_ij`.

        phase_ZZ_product = 1.0
        for i in range(n_spatial_qubits):
            for j in range(i): # i > j condition, so j goes from 0 to i-1
                # Qubit indices for j_i and j_j are (i+1) and (j+1) respectively (q_0 is s)
                j_i_bit = (k >> (i + 1)) & 1
                j_j_bit = (k >> (j + 1)) & 1

                beta_ij = theta_angle / (2**(2 * n_spatial_qubits - i - j))
                rz_angle_on_ancilla = -2 * beta_ij # Calculated RZ angle for this specific circuit decomposition

                # The effective state of the ancilla *during* the RZ gate application
                # This is 0 if j_i_bit == j_j_bit, and 1 if j_i_bit != j_j_bit.
                effective_ancilla_state_for_rz = (j_i_bit ^ j_j_bit) # XOR sum

                # The phase factor contributed by this single ZZ term's circuit
                # RZ(angle) applies exp(-i * angle/2 * Z) where Z on |0> is +1 and on |1> is -1
                phase_from_current_zz_term = np.exp(-1j * rz_angle_on_ancilla / 2 * ((-1)**effective_ancilla_state_for_rz))

                phase_ZZ_product *= phase_from_current_zz_term

        # Apply all calculated phases (global phase is ignored)
        new_amplitudes[k] *= phase_single_Z_product * phase_ZZ_product

    # Return the new Statevector with the updated amplitudes
    return Statevector(new_amplitudes)

In [29]:
class PK2dtGate(Gate):
    """
    Custom gate for the P(k^2 dt) operator.
    This gate applies the single Z phases and the ZZ phases via the
    specified CNOT-RZ-CNOT decomposition.
    """
    def __init__(self, phi_angle: float, theta_angle: float, n_spatial_qubits: int):
        self.n_spatial_qubits = n_spatial_qubits
        self.phi_angle = phi_angle
        self.theta_angle = theta_angle

        # Total qubits this gate acts on:
        # 1 (s) + n_spatial_qubits (j) + 1 (ancilla)
        # Quibit mapping (LSB to MSB): q_0: s, q_1...q_n: j_0...j_{n-1}, q_{n+1}: ancilla

        super().__init__('P(k^2dt)', n_spatial_qubits + 2, [], f"P(k^2dt)_n{n_spatial_qubits}")

    def _define(self):
        # We need to define the circuit operations inside this method.
        # This will be constructed as if it were a sub-circuit.

        qr_s = QuantumRegister(1, 's')
        qr_j = QuantumRegister(self.n_spatial_qubits, 'j')
        qr_anc = AncillaRegister(1, 'anc')

        # Create a sub-circuit for the gate definition.
        # The order of QRs here must match the order in which the gate's qubits are passed.
        # So it should be (qr_s, qr_j, qr_anc) to match the append order in build_step1_circuit
        sub_qc = QuantumCircuit(qr_s, qr_j, qr_anc, name='pk2dt_internal')

        # Map internal QR indices to the gate's flat qubit list
        s_qubit = qr_s[0]
        ancilla_qubit = qr_anc[0]

        # --- Apply single-qubit Z terms ---
        # Z_l acts on |j_l>, where j_l corresponds to qr_j[l].
        # Angle for Z_l is alpha_l = theta / 2^(2n-l-4)
        # RZ(angle) = exp(-i*angle*Z/2). To implement exp(i*A*Z) from Z acting on qubit,
        # we need RZ(-2*A).
        for l in range(self.n_spatial_qubits):
            j_l_qubit = qr_j[l] # j_l corresponds to index 'l' in the j register (e.g., qr_j[0] is j_0)
            alpha_l = self.theta_angle / (2**(2 * self.n_spatial_qubits - l - 4))
            sub_qc.rz(-2 * alpha_l, j_l_qubit)

        # --- Apply two-qubit ZZ terms using the specified decomposition ---
        # Z_i \otimes Z_j implemented as:
        # CNOT(j_i, ancilla) -> CNOT(j_j, ancilla) -> RZ(angle) on ancilla -> CNOT(j_j, ancilla) -> CNOT(j_i, ancilla)
        # Angle for RZ is -2 * beta_ij, where beta_ij = theta / 2^(2n-i-j)

        for i in range(self.n_spatial_qubits):
            for j in range(i): # iterate j from 0 to i-1
                j_i_qubit = qr_j[i]
                j_j_qubit = qr_j[j]

                beta_ij = self.theta_angle / (2**(2 * self.n_spatial_qubits - i - j))
                rz_angle_on_ancilla = -2 * beta_ij

                sub_qc.cx(j_i_qubit, ancilla_qubit)
                sub_qc.cx(j_j_qubit, ancilla_qubit)
                sub_qc.rz(rz_angle_on_ancilla, ancilla_qubit) # RZ gate on the ancilla
                sub_qc.cx(j_j_qubit, ancilla_qubit)
                sub_qc.cx(j_i_qubit, ancilla_qubit)

        self.definition = sub_qc

# --- 2. Define P(V_F delta t) as a Custom Gate ---
# This operator is diagonal and depends on the specific V_F function.
# For a generic V_F, its circuit decomposition involves multi-controlled phase gates.
# We'll create a placeholder custom gate for it.
class PVFdtGate(Gate):
    """
    Custom gate for the P(V_F*delta_t) operator, where V_F is a LINEAR potential V_F(x) = C_linear * x.
    This gate applies phases exp(-i * delta_t / h_reduced * V_F(x_j)).
    It decomposes into RZ gates on the spatial qubits (j_l) and sets a global phase.
    """
    def __init__(self, C_linear: float, delta_t: float, h_reduced: float, d: float, n_spatial_qubits: int):
        self.C_linear = C_linear
        self.delta_t = delta_t
        self.h_reduced = h_reduced
        self.d = d
        self.n_spatial_qubits = n_spatial_qubits

        # This gate acts on all spatial qubits (j_0 to j_{n-1}) and the 's' qubit (q_0).
        # The 's' qubit does not influence the phase, but it's part of the register.
        # It does NOT use the ancilla qubit from the main circuit.
        super().__init__('P(V_Fdt)', n_spatial_qubits + 1, [], f"P(V_Fdt)_linear_n{n_spatial_qubits}")

    def _define(self):
        qr_s = QuantumRegister(1, 's')
        qr_j = QuantumRegister(self.n_spatial_qubits, 'j')

        sub_qc = QuantumCircuit(qr_s, qr_j, name='pvfdt_linear_internal')

        # Calculate delta_x from the domain discretization
        delta_x = (2 * self.d) / (2**self.n_spatial_qubits)

        # The phase applied to each state |j> is exp(-i * (delta_t / h_reduced) * V_F(x_j))
        # V_F(x_j) = C_linear * x_j
        # x_j = -d + (j + 0.5) * delta_x
        # Exponent = -i * (delta_t / h_reduced) * C_linear * (-d + (j + 0.5) * delta_x)
        # Exponent = -i * (delta_t * C_linear / h_reduced) * (-d + 0.5 * delta_x)  (Global part)
        #            -i * (delta_t * C_linear / h_reduced) * delta_x * j           (j-dependent part)

        # Coefficient for the j-dependent part: lambda = - (delta_t * C_linear / h_reduced) * delta_x
        lambda_coeff = - (self.delta_t * self.C_linear / self.h_reduced) * delta_x

        # Implement exp(i * lambda_coeff * j) using RZ gates.
        # Since j = sum(j_k * 2^k), this is Prod_k exp(i * lambda_coeff * j_k * 2^k)
        # For each qubit j_k, if j_k=1, a phase of exp(i * lambda_coeff * 2^k) is applied.
        # Qiskit's RZ(theta) gate applies exp(-i * theta/2 * Z).
        # We want to apply a phase 'phi' for the |1> state using RZ.
        # The matrix for RZ(theta) is: [[exp(-i*theta/2), 0], [0, exp(i*theta/2)]]
        # We want exp(i*phase_for_1) for |1> state. So, theta/2 = phase_for_1 => theta = 2*phase_for_1
        # The phase_for_1 here is (lambda_coeff * 2^k)
        # So, the RZ angle is 2 * (lambda_coeff * 2^k)
        for k in range(self.n_spatial_qubits):
            j_k_qubit = qr_j[k]
            # Angle for RZ on qubit j_k
            rz_angle = 2 * (lambda_coeff * (2**k))
            sub_qc.rz(rz_angle, j_k_qubit)

        # Add the global phase component
        # This part comes from - (delta_t * C_linear / h_reduced) * (-d + 0.5 * delta_x)
        gamma_constant_part = - (self.delta_t * self.C_linear / self.h_reduced)
        global_phase_angle = gamma_constant_part * (-self.d + 0.5 * delta_x)

        # Set the global phase of the sub-circuit
        sub_qc.global_phase = global_phase_angle

        self.definition = sub_qc


2. Step 2,3,4: Normalization and Gauge transformation

In [30]:
def apply_un_normalization_to_statevector(
    current_statevector: Statevector,
    n_spatial_qubits: int
) -> Statevector:
    """
    Applies the conceptual U_N normalization operation directly to a Qiskit Statevector.
    This function normalizes the |s> qubit amplitudes for each distinct |j> (spatial)
    subspace based on the *current* statevector's amplitudes, as required by Eq. (49).

    Args:
        current_statevector (Statevector): The input statevector of the system (|s> |j> ... |ancilla>).
        n_spatial_qubits (int): The number of qubits for spatial discretization.

    Returns:
        Statevector: The new Statevector after applying the U_N normalization.
    """
    new_amplitudes = np.copy(

    )
    total_num_qubits = current_statevector.num_qubits

    # Determine the number of auxiliary qubits (if any) in addition to |s> and |j>
    # In the provided setup: 1 (s) + n_spatial_qubits (j) + 1 (ancilla)
    # So, total_num_qubits = 1 + n_spatial_qubits + num_ancilla_qubits
    # Here, num_ancilla_qubits = total_num_qubits - (1 + n_spatial_qubits)
    num_other_qubits = total_num_qubits - (1 + n_spatial_qubits) # This refers to the ancilla qubit in our context

    # The qubit ordering is assumed to be LSB = s, then j_0, ..., j_{n-1}, then other (ancilla) qubits as MSBs.
    # So, an index `k` in the statevector corresponds to |other ... j_{n-1}...j_0 s>.

    num_j_states = 2**n_spatial_qubits

    # Iterate through each possible combination of 'other_qubits' (e.g., ancilla) and 'j' states
    # For each |other, j> pair, we will normalize the |s> amplitudes.
    for other_val_decimal in range(2**num_other_qubits):
        for j_idx in range(num_j_states):
            # Calculate the full statevector index for |s=0, j_idx, other_val_decimal> and |s=1, j_idx, other_val_decimal>
            # Index format: |other_val_decimal (MSB) | j_idx (middle) | s_val (LSB) >
            # Example for n=2, 1 ancilla: |ancilla j1 j0 s>
            # Index for |s=0, j_idx, other_val_decimal>:
            # (other_val_decimal << (n_spatial_qubits + 1)) | (j_idx << 1) | 0
            index_s0 = (other_val_decimal << (n_spatial_qubits + 1)) | (j_idx << 1) | 0

            # Index for |s=1, j_idx, other_val_decimal>:
            # (other_val_decimal << (n_spatial_qubits + 1)) | (j_idx << 1) | 1
            index_s1 = (other_val_decimal << (n_spatial_qubits + 1)) | (j_idx << 1) | 1

            # Get the current amplitudes for |s=0, j> and |s=1, j> for this 'other' state
            amp_s0_current = new_amplitudes[index_s0]
            amp_s1_current = new_amplitudes[index_s1]

            # Calculate the normalization factor based on these current amplitudes
            norm_sq = np.abs(amp_s0_current)**2 + np.abs(amp_s1_current)**2

            if np.isclose(norm_sq, 0.0):
                # If both amplitudes are zero, no normalization is needed/possible.
                # The state remains zero in this subspace.
                continue

            norm_factor = np.sqrt(norm_sq)

            # Apply the normalization by dividing the current amplitudes by the norm factor
            new_amplitudes[index_s0] /= norm_factor
            new_amplitudes[index_s1] /= norm_factor

    # The overall statevector might lose its global normalization of 1 after these local normalizations,
    # as the sum of |Psi_s(x_j,t)|^2 over j and s might change.
    # Therefore, a final global normalization of the *entire* statevector is usually appropriate
    # after applying such local normalization steps if it's meant to represent a valid quantum state.
    # However, the problem statement only implies local normalization `Q_j`.
    # I will return the state after local normalizations, and suggest a global normalize in example.

    return Statevector(new_amplitudes)

In [31]:
#Step 3, 4: depend on the setting, q is the solution of the Possion equation:
#\partial^2_x q  = h/2 (\partial^2_x \overline{\psi^**}i \psi^{**} -\overline{\psi^**} i \partial^2_x \psi^**)
class PQxGate(Gate):
    """
    Custom gate for the P(q(x_j)) diagonal unitary transformation.
    This gate applies a phase e^{-iq(x_j)/h_reduced} to each computational basis state |j>
    of the spatial qubits.
    """
    def __init__(self, q_function, d: float, n_spatial_qubits: int, h_reduced: float):
        self.q_function = q_function
        self.d = d
        self.n_spatial_qubits = n_spatial_qubits
        self.h_reduced = h_reduced

        super().__init__('P(q(x))', n_spatial_qubits, [], f"P(q(x))_n{n_spatial_qubits}")

    def _define(self):
        qr_j = QuantumRegister(self.n_spatial_qubits, 'j')
        sub_qc = QuantumCircuit(qr_j, name='pqx_internal')

        x_points, _ = discretize_domain_with_binary_vectors(self.d, self.n_spatial_qubits)

        num_j_states = 2**self.n_spatial_qubits
        diagonal_phases = []

        # Calculate the phase for each |j> state
        for j_idx in range(num_j_states):
            x_j_current = x_points[j_idx]

            # Calculate q(x_j)
            q_val = self.q_function(x_j_current)

            # Calculate the phase angle: -q(x_j) / h_reduced
            phase_angle = -q_val / self.h_reduced

            # The diagonal entry is e^(i * phase_angle)
            diagonal_phases.append(np.exp(1j * phase_angle))

        # Create a DiagonalGate from the calculated phases
        diagonal_gate = DiagonalGate(diagonal_phases)

        # Apply the DiagonalGate to the spatial qubits
        sub_qc.append(diagonal_gate, qr_j[:])

        self.definition = sub_qc

In [39]:
def build_full_quantum_step(
    n_spatial_qubits: int,
    C_linear_vf: float, delta_t: float, h_reduced: float, d: float,
    phi_angle_pk2dt: float, theta_angle_pk2dt: float,
    q_function # The function q(x) for P(q(x))
) -> QuantumCircuit:
    """
    Constructs a full quantum circuit segment combining previous steps and the new P(q(x)) gate.

    Qubit Mapping (LSB to MSB, consistent with previous code):
    q_0: 's' qubit
    q_1 to q_n_spatial_qubits: 'j_0' to 'j_{n_spatial_qubits-1}' qubits (j-register)
    q_{n_spatial_qubits + 1}: 'ancilla' qubit (ancilla register)
    """
    qr_s = QuantumRegister(1, 's')
    qr_j = QuantumRegister(n_spatial_qubits, 'j')
    qr_anc = AncillaRegister(1, 'anc')

    qc = QuantumCircuit(qr_s, qr_j, qr_anc)

    # 1. P(V_F dt) operator
    pvfdt_gate = PVFdtGate(C_linear_vf, delta_t, h_reduced, d, n_spatial_qubits)
    qc.append(pvfdt_gate, [qr_s[0]] + qr_j[:])

    # 2. QFT on |j_0>,...,|j_{n-1}>
    qc.append(QFT(n_spatial_qubits), qr_j)

    # 3. P(k^2 dt) operator
    pk2dt_gate = PK2dtGate(phi_angle_pk2dt, theta_angle_pk2dt, n_spatial_qubits)
    qc.append(pk2dt_gate, [qr_s[0]] + qr_j[:] + [qr_anc[0]])

    # 4. Inverse QFT on |j_0>,...,|j_{n-1}>
    qc.append(QFT(n_spatial_qubits).inverse(), qr_j)

    # 5. P(q(x_j)) operator (newly added)
    # This gate acts *only* on the spatial qubits (j register).
    pqx_gate = PQxGate(q_function, d, n_spatial_qubits, h_reduced)
    qc.append(pqx_gate, qr_j[:])

    return qc


In [41]:
if __name__ == '__main__':
    # Define some placeholder parameters for demonstration
    n_spatial_qubits_example = 2 # For j_0, j_1. Total spatial points = 2^2 = 4.

    C_linear_example = 0.5 # Example constant for the linear potential V_F

    delta_t_example = 0.1 # Time step
    h_reduced_example = 1.0 # Reduced Planck constant
    d_example = 10.0 # Half-width of the domain [-d, d]

    phi_angle_pk2dt_example = np.pi / 4
    theta_angle_pk2dt_example = np.pi / 2

    # Define the function q(x) for P(q(x_j))
    def example_q_function(x):
        # A simple quadratic potential for demonstration
        return 0.1 * x**2 + 0.5 * x

    # Build the complete quantum step circuit
    full_step_circuit = build_full_quantum_step(
        n_spatial_qubits_example,
        C_linear_example, delta_t_example, h_reduced_example, d_example,
        phi_angle_pk2dt_example, theta_angle_pk2dt_example,
        example_q_function
    )

    print("--- Full Quantum Step Circuit (including P(q(x))) ---")
    print(full_step_circuit.draw(output='text', fold=-1))
    print(f"\nTotal qubits in circuit: {full_step_circuit.num_qubits}")
    print(f"Number of spatial qubits (n): {n_spatial_qubits_example}")
    print(f"Qubit mapping: q[0]=s, q[1]=j_0, q[2]=j_1, q[3]=ancilla (for n=2 example)")

    print("\n--- Expanded P(q(x)) Gate Definition ---")
    pqx_example_gate = PQxGate(example_q_function, d_example, n_spatial_qubits_example, h_reduced_example)
    print(pqx_example_gate.definition.draw(output='text', fold=-1))

    # --- Example of statevector simulation for P(q(x)) ---
    print("\n--- Statevector Simulation Example for P(q(x)) ---")

    # A simple dummy function for psi_coefficients_map for initial state
    def simple_psi_coefficients_map(s_val, x_j_val, time_t):
        if s_val == 0:
            return np.exp(-(x_j_val / 5.0)**2) # Gaussian
        else:
            return 0.0 + 0.0j # Only s=0 part for simplicity

    # Construct an initial statevector. The Statevector constructor will handle normalization.
    initial_state = construct_quaternionic_psi_statevector(
        d_example, n_spatial_qubits_example, simple_psi_coefficients_map, 0.0
    )

    print(f"Initial Statevector (norm = {initial_state.norm():.4f}):")
    print(initial_state.data)

    # To isolate the effect of P(q(x)), let's create a temporary circuit with just this gate
    qc_pqx_only = QuantumCircuit(initial_state.num_qubits) # Use initial_state's num_qubits

    # The PQxGate acts only on the j-register (qubits 1 to n_spatial_qubits_example in our mapping)
    # We need to ensure the target qubits for `append` are correct.
    # The `j` qubits start from index 1.
    j_qubits = list(range(1, 1 + n_spatial_qubits_example))

    pqx_gate_instance = PQxGate(example_q_function, d_example, n_spatial_qubits_example, h_reduced_example)
    qc_pqx_only.append(pqx_gate_instance, j_qubits)

    # Simulate applying only P(q(x)) to the initial state
    state_after_pqx = initial_state.evolve(qc_pqx_only)
    # No explicit .normalize() call needed here as evolve() of a unitary gate preserves norm.

    print(f"\nStatevector after P(q(x)) gate (norm = {state_after_pqx.norm():.4f}):")
    print(state_after_pqx.data)

    # Verify a specific phase change for a component
    # For example, consider the amplitude for |s=0, j=0, anc=0> (index 0)
    # Original amplitude for |000> (s=0, j=0, anc=0)
    # This corresponds to x_j for j=0, which is -d + 0.5 * delta_x
    x_points, _ = discretize_domain_with_binary_vectors(d_example, n_spatial_qubits_example)
    x_j0 = x_points[0]

    initial_amplitude_000 = initial_state.data[0]
    expected_q_val = example_q_function(x_j0)
    expected_phase_angle = -expected_q_val / h_reduced_example

    # The phase is applied only to the 'j' part. The 's' part is not touched.
    # The amplitude of |ancilla j s> will be scaled by this phase.

    print(f"\nChecking phase for state |000> (s=0, j=0, anc=0):")
    print(f"  x_j for j=0: {x_j0}")
    print(f"  q(x_j0): {expected_q_val}")
    print(f"  Expected phase angle: {expected_phase_angle:.4f} radians")

    # The original amplitude of |000> is `initial_state.data[0]`
    # The expected new amplitude should be `initial_state.data[0] * exp(i * expected_phase_angle)`
    expected_amplitude_000_after_pqx = initial_amplitude_000 * np.exp(1j * expected_phase_angle)

    print(f"  Initial amplitude |000>: {initial_amplitude_000}")
    print(f"  Expected amplitude |000> after PQ(x): {expected_amplitude_000_after_pqx}")
    print(f"  Actual amplitude |000> after PQ(x): {state_after_pqx.data[0]}")

    # Check if the phases match (after accounting for potential global normalization differences if any)
    # We can check the complex argument (angle)
    initial_angle = np.angle(initial_amplitude_000)
    final_angle = np.angle(state_after_pqx.data[0])

    observed_phase_change = final_angle - initial_angle
    # Wrap to [-pi, pi]
    observed_phase_change = (observed_phase_change + np.pi) % (2 * np.pi) - np.pi

    print(f"  Observed phase change for |000>: {observed_phase_change:.4f} radians")
    print(f"  Difference (Observed - Expected): {observed_phase_change - expected_phase_angle:.6f}")
    assert np.isclose(observed_phase_change, expected_phase_angle), "Phase did not match expected!"
    print("  Phase matches expected!")


--- Full Quantum Step Circuit (including P(q(x))) ---
     ┌─────────────────────┐        ┌──────────────┐                        
  s: ┤0                    ├────────┤0             ├────────────────────────
     │                     │┌──────┐│              │┌───────┐┌─────────────┐
j_0: ┤1 P(V_Fdt)_linear_n2 ├┤0     ├┤1             ├┤0      ├┤0            ├
     │                     ││  QFT ││  P(k^2dt)_n2 ││  IQFT ││  P(q(x))_n2 │
j_1: ┤2                    ├┤1     ├┤2             ├┤1      ├┤1            ├
     └─────────────────────┘└──────┘│              │└───────┘└─────────────┘
anc: ───────────────────────────────┤3             ├────────────────────────
                                    └──────────────┘                        

Total qubits in circuit: 4
Number of spatial qubits (n): 2
Qubit mapping: q[0]=s, q[1]=j_0, q[2]=j_1, q[3]=ancilla (for n=2 example)

--- Expanded P(q(x)) Gate Definition ---
     ┌───────────────────────────────────────────────────────────────────────

/tmp/ipython-input-39-2493002813.py:26: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_spatial_qubits), qr_j)
/tmp/ipython-input-39-2493002813.py:33: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_spatial_qubits).inverse(), qr_j)


AttributeError: 'Statevector' object has no attribute 'norm'